# 인터파크

In [1]:
import requests
from urllib.parse import quote
import pandas as pd
from bs4 import BeautifulSoup

- 순위, 책제목, 저자, 출판사, 가격(정수), 발행일, 쪽수(정수)

### 1. 데이터 가져오기 

In [2]:
site = 'http://book.interpark.com'
url = 'http://book.interpark.com/display/collectlist.do?_method=bestsellerHourNew&bookblockname=b_gnb&booklinkname=%BA%A3%BD%BA%C6%AE%C1%B8&bid1=w_bgnb&bid2=LiveRanking&bid3=main&bid4=001'
req = requests.get(url)
html = req.text

In [3]:
soup = BeautifulSoup(html, 'html.parser')

### 2. 찾으려는 데이터의 태그 찾기

In [4]:
lis = soup.select_one('.rankBestContentList ').find_all('li')
len(lis)


69

In [5]:
lis = soup.select('.rankBestContentList > ol > li' )
len(lis)


15

### 3. 하나를 선택해서 원하는 정보추출

- 순위, 제목, 저자, 출판사, 가격(정수), 발행일, 쪽수(정수)

- 트렌드 코리아 2022

In [6]:
li = lis[0]
href = li.find('a')['href']
href
# href이 뭘 위한 건지 잘 모르겠다. 



'/product/BookDisplay.do?_method=detail&sc.shopNo=0000400000&sc.prdNo=354069935&sc.saNo=003003001&bid1=Best_zone&bid2=LiveRanking&bid3=PRD&bid4=001'

In [14]:
sub_url = site + href 
sub_url

'http://book.interpark.com/product/BookDisplay.do?_method=detail&sc.shopNo=0000400000&sc.prdNo=354069935&sc.saNo=003003001&bid1=Best_zone&bid2=LiveRanking&bid3=PRD&bid4=001'

In [15]:
sub_req = requests.get(sub_url)
sub_html = sub_req.text
sub_soup = BeautifulSoup(sub_html, 'html.parser')

In [9]:
title = li.select_one('.itemName').get_text()
title


'트렌드 코리아 2022 '

In [10]:
author = li.select_one('.author').get_text()
author

'김난도(金蘭都), 전미영, 최지혜, 이향은, 이준영'

In [11]:
pub = li.select_one('.company').get_text()
pub

'미래의창'

In [85]:
price = int(li.select_one('.price').find('em').string.replace(',',''))
price

16200

### 4. 페이지에 있는 전체 데이터 가져오기

- with 제목, 저자, 출판사, 가격 

In [97]:
lines = []
for li in lis:
    title = li.select_one('.itemName').get_text()
    author = li.select_one('.author').get_text()
    pub = li.select_one('.company').get_text()
    price = int(li.select_one('.price').find('em').string.replace(',',''))
    
    lines.append([title, author, pub, price])

df = pd.DataFrame(lines, columns =['제목', '저자', '출판사', '가격'] )
df


,제목,저자,출판사,가격
0,트렌드 코리아 2022,"김난도(金蘭都), 전미영, 최지혜, 이향은, 이준영",미래의창,16200
1,거인의 포트폴리오,강환국,페이지2북스,16200
2,2022 현직교사들이 들려주는 면접레시피,"류은진, 양왕경, 이광한, 이지혜, 양왕경, 이광한, 이지혜",미래가치,42300
3,거꾸로 읽는 세계사,유시민(柳時敏),돌베개,15750
4,아직 오지 않은 날들을 위하여,파스칼 브뤼크네르/이세진 역,인플루엔셜,14400
5,작은 별이지만 빛나고 있어(10만 부 기념 한정판 에디션),소윤,북로망스,13500
6,투자는 디테일에 있다,김정환,부케이,16200
7,웰씽킹(Wealthinking),켈리 최,다산북스,14400
8,오십에 읽는 논어,최종엽,유노북스,14400
9,소크라테스 익스프레스(20만 부 기념 윈터 에디션),에릭 와이너/김하현 역,어크로스,16200


# 순위, 발행일, 쪽수(정수) 보류 

In [42]:
lis[0].select_one('.rankBtn_ctrl')['class']     # class 속성 

['rankBtn_ctrl', 'rkNum_B01']

In [38]:
lis[12].select('.rankBtn_ctrl')

[<span class="rankBtn_ctrl rkNum_M01"></span>,
 <span class="rankBtn_ctrl rkNum_M03"></span>]

In [45]:
lis[12].select('.rankBtn_ctrl')[0]['class']

['rankBtn_ctrl', 'rkNum_M01']

In [ ]:
if len(li.select('.rankBtn_ctrl')) == 1:
    rank = soup.select_one('')
    else:
        rank = 

In [105]:
date = sub_soup.select_one('.bInfo_txt').get_text().split()
date

['저',
 ':',
 '김난도,',
 '전미영,',
 '최지혜,',
 '이향은,',
 '이준영',
 '출판사',
 ':',
 '미래의창',
 '발행',
 ':',
 '2021년',
 '10월',
 '06일',
 '쪽수',
 ':',
 '452',
 'ISBN',
 ':',
 '9788959897001']

In [104]:
date = sub_soup.select_one('.bInfo_txt').get_text().split()[12:15]
date


['2021년', '10월', '06일']

In [33]:
date = sub_soup.select_one('.bInfo_txt')
date

KeyError: 1

In [68]:
page = sub_soup.select('.tb_changeRefund > tbody > tr')
page

[]

In [72]:
page = sub_soup.select_one('.bInfo_txt').get_text()
len(page)

89